In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(1,6,3)
        self.conv2=nn.Conv2d(6,16,3)
        #y=Wx+b
        self.fc1=nn.Linear(16*6*6,120) #6x6 img size
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1, self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
    
net=Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [20]:
i=0
for p in params:
    i=i+1
    print (i)
    print(p)
    

1
Parameter containing:
tensor([[[[-0.0251,  0.2904, -0.1043],
          [-0.2975,  0.2470,  0.1396],
          [-0.1516, -0.2306,  0.2652]]],


        [[[-0.2740, -0.2286, -0.0505],
          [-0.2362, -0.2883,  0.0128],
          [-0.2023, -0.2806, -0.1493]]],


        [[[-0.1953, -0.2508,  0.1211],
          [ 0.2831,  0.1008,  0.1108],
          [-0.2136, -0.1163, -0.0090]]],


        [[[ 0.2632,  0.2141, -0.2929],
          [-0.0904, -0.2230,  0.1524],
          [-0.1944, -0.1887,  0.2238]]],


        [[[-0.3323,  0.1432, -0.2140],
          [ 0.3140,  0.1027,  0.0909],
          [-0.2667,  0.1089, -0.2926]]],


        [[[-0.1293,  0.1181,  0.0385],
          [-0.1706, -0.3104, -0.1685],
          [ 0.0345, -0.3247,  0.3206]]]], requires_grad=True)
2
Parameter containing:
tensor([ 0.2562, -0.2586,  0.2464,  0.2424,  0.2751, -0.1919],
       requires_grad=True)
3
Parameter containing:
tensor([[[[-1.0383e-01, -1.2423e-01, -3.5542e-02],
          [ 2.4876e-02,  1.1296e-01, -1.63

          [ 8.3174e-02, -6.6677e-02,  1.3020e-01]]]], requires_grad=True)
4
Parameter containing:
tensor([-0.1039, -0.1193,  0.0922, -0.1154,  0.1232,  0.0552, -0.1248,  0.0691,
        -0.0714, -0.0787, -0.0470, -0.0145,  0.1087,  0.0406, -0.1315,  0.1122],
       requires_grad=True)
5
Parameter containing:
tensor([[ 0.0339, -0.0146, -0.0256,  ..., -0.0187,  0.0171, -0.0195],
        [-0.0389, -0.0228, -0.0143,  ...,  0.0255, -0.0100, -0.0175],
        [-0.0132,  0.0236, -0.0152,  ...,  0.0135, -0.0173,  0.0187],
        ...,
        [ 0.0135,  0.0363,  0.0089,  ..., -0.0286,  0.0341,  0.0148],
        [ 0.0296,  0.0059, -0.0334,  ...,  0.0371,  0.0152,  0.0376],
        [-0.0106, -0.0376,  0.0401,  ..., -0.0101,  0.0412,  0.0407]],
       requires_grad=True)
6
Parameter containing:
tensor([ 0.0168, -0.0259,  0.0151, -0.0156, -0.0096,  0.0194, -0.0243,  0.0193,
         0.0206,  0.0234,  0.0372, -0.0379, -0.0311, -0.0187, -0.0164,  0.0265,
        -0.0141,  0.0275, -0.0261, -0.0144,  

          0.0648, -0.1068, -0.0558, -0.0486]], requires_grad=True)
10
Parameter containing:
tensor([ 0.0724, -0.0943, -0.0420, -0.0756, -0.0664, -0.0571, -0.0076, -0.0936,
         0.0188,  0.0704], requires_grad=True)


In [23]:
input=torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[ 0.1173, -0.0467,  0.0336,  0.0043,  0.1249,  0.0993, -0.0287,  0.1073,
         -0.1602, -0.0188]], grad_fn=<AddmmBackward>)


In [24]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [25]:
output=net(input)
target=torch.randn(10)# dummy target
target=target.view(1,-1)
criterion=nn.MSELoss()

loss=criterion(output, target)
print(loss)

tensor(1.0545, grad_fn=<MseLossBackward>)


In [29]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])#linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu


In [30]:
net.zero_grad()
print('conv1.bias.grad before backprop')
print (net.conv1.bias.grad)

loss.backward()
print ('conv1.bias.grad after backprop')
print(net.conv1.bias.grad)

conv1.bias.grad before backprop
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backprop
tensor([ 0.0261,  0.0187, -0.0014,  0.0142, -0.0100, -0.0026])


In [31]:
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [32]:
import torch.optim as optim
#create optimizer
optimizer=optim.SGD(net.parameters(),lr=0.01)

#in training loop:
optimizer.zero_grad() #zero grad buffer
output=net(input)
loss=criterion(output, target)
loss.backward()
optimizer.step() #does update
